#### Import libraries and setup matplotlib

In [ ]:
import matplotlib
%matplotlib inline
import matplotlib.pylab as plt

import IPython.display as ipd

import sys
import numpy as np
import torch

from hparams import create_hparams
from model import Tacotron2
from layers import TacotronSTFT, STFT
from audio_processing import griffin_lim
from train import load_model
from text import text_to_sequence
#from denoiser import Denoiser

#my libraries
import scipy.io
from phoneme_to_seq import *
from fastdtw import fastdtw
import random
from scipy.spatial.distance import euclidean
from scipy.stats import pearsonr

In [ ]:
def plot_data(data, figsize=(16, 4)):
    fig, axes = plt.subplots(1, len(data), figsize=figsize)
    for i in range(len(data)):
        axes[i].imshow(data[i], aspect='auto', origin='bottom', 
                       interpolation='none')

#### Setup hparams

In [ ]:
hparams = create_hparams()
hparams.sampling_rate = 22050

#### Load model from checkpoint

In [ ]:
checkpoint_path = "..."
print("loading: \n",checkpoint_path)
model = load_model(hparams)
model.load_state_dict(torch.load(checkpoint_path)['state_dict'])
_ = model.cuda().eval().half()

#### INFERENCE

In [ ]:
dct=prep_dct()
test_file= open("path to meta file for testing",'r')
print("Testin on : ",test_file.name)
lines=test_file.readlines()
coefficients=[]
rMSE=[]
for line in lines:
    path=line.split('|')
        
    sequence = np.array(to_sequence(path[1][:-1],dct))[None, :]
    sequence = torch.autograd.Variable(
        torch.from_numpy(sequence)).cuda().long()
    torch.manual_seed(1234)
    mel_outputs, mel_outputs_postnet, _, alignments = model.inference(sequence)

    X=clip_ema_silence(path[0])
    Y=mel_outputs_postnet[0].cpu().detach().numpy().transpose()
    #print(X.shape,Y.shape)
    dis,pth=fastdtw(X,Y, dist=euclidean)
    for artic in range(0,12):
        out=[]
        gt=[]
        for i in range(0,len(pth)):
            out.append(Y[pth[i][1]][artic])
            #for i in range(0,Yout.shape[0]):
            gt.append(X[pth[i][0]][artic])
        coef=pearsonr(out,gt)
        coefficients.append(coef)
        rMSE.append(np.sqrt(np.mean(np.square(np.asarray(out)-np.asarray(gt)))))
print(sub)
print(np.mean(rMSE,axis=0))      
print(np.mean(coefficients,axis=0))

<h4>CORRELATION AND RMSE for 12 articulators</h4>

In [ ]:
c=np.asarray(coefficients)
c=c[:,0].reshape(46,12)
c=np.mean(c,axis=0)
print("Correlation : \n",c)
rmse=np.asarray(rMSE)
rmse=rmse[:].reshape(46,12)
rmse=np.mean(rmse,axis=0)
print("RMSE : \n", rmse)

<h4>Store results in excel</h4>

In [ ]:
##write xlsx
import xlsxwriter
workbk=xlsxwriter.Workbook('excel_sheets/results_'+sub_list[sub]+'.xlsx')
wrksht=workbk.add_worksheet()
print("Writing to ",'excel_sheets/results_'+sub_list[sub]+'.xlsx')
row=0
col=0
for x,y,i in zip(c,rmse,range(0,12)):
    wrksht.write(row,col,x)
    wrksht.write(row+5,col,y)
    col+=1
workbk.close()

<h3>Best Articulatory Plots</h3

In [ ]:
coeffi=[]
path=lines[int(coefficients.index(max(coefficients))/12)].split('|')
sequence = np.array(to_sequence(path[1][:-1],dct))[None, :]
sequence = torch.autograd.Variable(torch.from_numpy(sequence)).cuda().long()
##### Decode text input and plot results
torch.manual_seed(1234)
mel_outputs, mel_outputs_postnet, _, alignments = model.inference(sequence)
#####plot with ema gt
####plot with dtw aligned and correlation
X=clip_ema_silence(path[0])
Y=mel_outputs[0].cpu().detach().numpy().transpose()
dis,pth=fastdtw(X,Y, dist=euclidean)
for artic in range(0,12):
    out=[]
    gt=[]
    for i in range(0,len(pth)):
        out.append(Y[pth[i][1]][artic])
        #for i in range(0,Yout.shape[0]):
        gt.append(X[pth[i][0]][artic])
    coef=pearsonr(out,gt)
    coeffi.append(coef)
    plt.plot(gt)
    plt.plot(out)
    plt.figure()
print(coeffi[coefficients.index(max(coefficients))%12])
print(np.mean(coeffi,axis=0))

<h1> Alignment for sentence</h1>

In [ ]:
dct=prep_dct()
path='...'
sequence = np.array(to_sequence(path,dct))[None, :]
sequence = torch.autograd.Variable(torch.from_numpy(sequence)).cuda().long()

##### Decode text input and plot results

torch.manual_seed(1234)
mel_outputs, mel_outputs_postnet, _, alignments = model.inference(sequence)
print(sequence.shape)
print(alignments.shape)
plot_data((mel_outputs.float().data.cpu().numpy()[0],
           mel_outputs_postnet.float().data.cpu().numpy()[0],
           alignments.float().data.cpu().numpy()[0].T))
